## <span style="color:blue">Group 18 Phase 4 Project Submission</span>

# CURRENCY EXCHANGE RATE PREDICTION
<span style="color:blue">**Students names:**</span>
1. Francis Njenga
2. Thomas Okiwi
2. Leonard Gachimu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import plotly.express as px


import warnings

# Ignore documentation warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sphinx')

In [2]:
import pandas as pd

In [3]:
import seaborn as sns

## Business Understanding

In the dynamic realm of forex trading, delving into the intricacies of analyzing BTC/USD for a data science project requires a blend of technical prowess and market intuition. At its core, a forex company's success hinges on its ability to decipher patterns, derive insights, and make informed decisions in the volatile landscape of cryptocurrency.

To embark on this data science journey, one must first acknowledge the unique nature of BTC/USD trading. Unlike traditional fiat currencies, Bitcoin operates in a decentralized ecosystem, susceptible to a myriad of factors ranging from market sentiment to regulatory developments. As such, any comprehensive analysis must transcend traditional forex metrics and incorporate the idiosyncrasies of the cryptocurrency market.

Fundamental analysis is a key pillar in understanding the dynamics of BTC/USD. Factors such as macroeconomic indicators, regulatory shifts, and technological advancements play a pivotal role in shaping Bitcoin's value. The global economic landscape, inflationary pressures, and institutional adoption are variables that demand meticulous examination.

On the technical front, deploying advanced algorithms and machine learning models becomes imperative. Time series analysis, sentiment analysis, and pattern recognition algorithms can unravel hidden trends within the price movements of BTC/USD. As cryptocurrencies often exhibit distinct behavioral patterns, these models can offer a predictive edge, allowing the forex company to stay ahead of market fluctuations.

Risk management is another critical facet that cannot be understated. The inherent volatility of BTC/USD necessitates robust risk mitigation strategies. Utilizing data-driven insights, a forex company can optimize position sizing, set stop-loss levels, and establish risk-reward ratios tailored to the unique characteristics of the cryptocurrency market.

Furthermore, the integration of real-time data feeds and APIs is paramount for staying abreast of market developments. In the fast-paced world of forex, timely and accurate information can make the difference between seizing an opportunity and incurring a loss.

In conclusion, a profound business understanding of analyzing BTC/USD for a data science project in the context of a forex company demands a holistic approach. By amalgamating fundamental insights, technical acumen, and cutting-edge data science methodologies, the company can navigate the complexities of the cryptocurrency market, ultimately fostering informed decision-making and sustainable success.

In [4]:
stock= pd.read_csv("BTC_USD_Historical_Data.csv")
stock.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,11/29/2023,"38,046.70","37,823.30","38,362.90","37,686.50",67.23K,0.59%
1,11/28/2023,"37,823.30","37,244.30","38,379.40","36,881.10",57.50K,1.54%
2,11/27/2023,"37,248.60","37,451.80","37,563.30","36,751.50",45.24K,-0.54%
3,11/26/2023,"37,451.80","37,786.40","37,819.10","37,166.30",29.20K,-0.89%
4,11/25/2023,"37,787.00","37,718.60","37,887.40","37,599.90",16.09K,0.18%


In [5]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4883 entries, 0 to 4882
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      4883 non-null   object
 1   Price     4883 non-null   object
 2   Open      4883 non-null   object
 3   High      4883 non-null   object
 4   Low       4883 non-null   object
 5   Vol.      4877 non-null   object
 6   Change %  4883 non-null   object
dtypes: object(7)
memory usage: 267.2+ KB


In [6]:
# Number of missing values in dataframe
stock.isna().sum()

Date        0
Price       0
Open        0
High        0
Low         0
Vol.        6
Change %    0
dtype: int64

Since the values in volumn column are currently object type with irregular characters, it is not possible to compute measures of central tendency or plot a distribution of the values.
We therefore, cannot statitically determine what to replace the missing values with and since they are only 6 out of 4,883 rows, we chose to delete them.

In [7]:
# Delete rows with missing values
stock.dropna(inplace=True)

Inspect the value counts in volume column

In [8]:
stock['Vol.'].value_counts()

1.05M      6
1.04M      5
14.73K     5
1.18M      5
1.11M      4
          ..
956.52K    1
840.76K    1
747.71K    1
738.17K    1
0.08K      1
Name: Vol., Length: 4335, dtype: int64

The volume column has several unique values and we notice that they are ending with letters such as K and M to denote multiples of thousands, millions, and so on.
We will use a regex function to extract all the unique suffixes and one representative value per suffix. 

In [9]:
# Import regex
import re

# Extract rows with suffix letters using a regex expression
suffixes = stock['Vol.'].str.extract(r'(\D+)$')[0]

# Create a dictionary to store unique values for each suffix
unique_values = {}
for suffix in suffixes.unique():
    # Get values with the current suffix
    values_with_suffix = stock[stock['Vol.'].str.endswith(suffix)]['Vol.']
    # Get the first value for the suffix
    representative_value = values_with_suffix.iloc[0]
    unique_values[suffix] = representative_value

# Print the unique values for each suffix
for suffix, value in unique_values.items():
    print(f"Suffix {suffix}: {value}")
    
# Print rows without a suffix letter in the volume column
stock[~(stock['Vol.'].str.contains(r'(\D+)$'))]

Suffix K: 67.23K
Suffix M: 671.05M
Suffix B: 1.04B


C:\Users\USER PC\AppData\Local\Temp\ipykernel_15972\4104812573.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  stock[~(stock['Vol.'].str.contains(r'(\D+)$'))]


,Date,Price,Open,High,Low,Vol.,Change %


There are no values without a suffix letter. We will next strip the suffix letters, convert the remaining number to a float type and multiply it with the corresponding multiplier for the letter.

In [10]:
# Strip suffix letters from volume column and convert to respective numeric figure
for index, row in stock.iterrows():
    if 'K' in row['Vol.']:
        row_value = round(float(row['Vol.'].strip('K')) * 1000)
        stock.at[index, 'Vol.'] = row_value
    elif 'M' in row['Vol.']:
        row_value = round(float(row['Vol.'].strip('M')) * 1000000)
        stock.at[index, 'Vol.'] = row_value
    elif 'B' in row['Vol.']:
        row_value = round(float(row['Vol.'].strip('B')) * 1000000000)
        stock.at[index, 'Vol.'] = row_value       

In [11]:
# Confirm changes to volume column
stock

,Date,Price,Open,High,Low,Vol.,Change %
0,11/29/2023,"38,046.70","37,823.30","38,362.90","37,686.50",67230,0.59%
1,11/28/2023,"37,823.30","37,244.30","38,379.40","36,881.10",57500,1.54%
2,11/27/2023,"37,248.60","37,451.80","37,563.30","36,751.50",45240,-0.54%
3,11/26/2023,"37,451.80","37,786.40","37,819.10","37,166.30",29200,-0.89%
4,11/25/2023,"37,787.00","37,718.60","37,887.40","37,599.90",16090,0.18%
...,...,...,...,...,...,...,...
4878,7/22/2010,0.1,0.1,0.1,0.1,2160,0.00%
4879,7/21/2010,0.1,0.1,0.1,0.1,580,0.00%
4880,7/20/2010,0.1,0.1,0.1,0.1,260,0.00%
4881,7/19/2010,0.1,0.1,0.1,0.1,570,0.00%


In [12]:
stock['Change %'].str.extract(r'(%+)$')

,0
0,%
1,%
2,%
3,%
4,%
...,...
4878,%
4879,%
4880,%
4881,%
